# Import packages

In [9]:
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm, tqdm_notebook, tqdm_pandas
tqdm_notebook().pandas()

In [3]:
%time df = pd.read_csv("/Users/huangshanhongtao/Downloads/BT4222 Project/Answers_clean.csv")

<string>:2: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 22.6 s, sys: 1.65 s, total: 24.3 s
Wall time: 24.3 s


In [4]:
df.shape

(2014531, 8)

In [5]:
df = df[df.full_text.notnull()]

In [6]:
df.shape

(1978899, 8)

In [7]:
best_answers = df[df.Score >= 8]

In [8]:
best_answers.shape

(100570, 8)

In [9]:
worst_answers = df[df.Score <= -1]

In [10]:
worst_answers.shape

(33536, 8)

In [11]:
combined = best_answers.append(worst_answers, ignore_index=True).copy()
#combined = df.copy()

In [12]:
combined.shape

(134106, 8)

## Remove hyper links within the cleaned text

In [13]:
import re

def remove_link(text):
    text = re.sub(r"http\S+", '', text)
    return text

In [14]:
combined['cleaned'] = combined.full_text.apply(remove_link)

## Count number of words

In [15]:
from textstat.textstat import textstat

In [16]:
combined['no_of_words'] = combined.cleaned.apply(textstat.lexicon_count)

## Count number of sentences

In [17]:
combined['no_of_sent'] = combined.cleaned.apply(textstat.sentence_count)

## Drop useless columns

In [18]:
combined.drop(['Id','OwnerUserId','CreationDate','ParentId'], axis=1, inplace=True)

## Drop number_of_words <=2

In [19]:
combined = combined[combined.no_of_words >= 2]

In [20]:
combined.shape

(133314, 7)

## Drop number_of_sentences <= 2

In [21]:
combined = combined[combined.no_of_sent >=2]

In [22]:
combined.shape

(99093, 7)

In [23]:
combined[(combined.Score <= -1)].shape

(20981, 7)

In [24]:
combined[(combined.Score >= 0)].shape

(78112, 7)

# Get SpaCy object

In [10]:
import spacy

nlp = spacy.load('en')

In [14]:
#combined = pd.read_csv("/Users/huangshanhongtao/Downloads/BT4222 Project Data/combined_clean.csv")

In [69]:
combined['spacy'] = combined.cleaned.apply(nlp)

In [30]:
combined.head()

,Score,Body,full_text,is_code_present,cleaned,no_of_words,no_of_sent,spacy,textacy,summary,no_of_polysyllable_words,flesch_readability,sentiment,subjectivity,quantitative,descriptive,good_bad
0,13.0,"<p><a href=""http://svnbook.red-bean.com/"">Vers...",Version Control with SubversionA very good res...,0.0,Version Control with SubversionA very good res...,17,2,(),"Doc(20 tokens; ""Version Control with Subversio...","{'n_chars': 104, 'coleman_liau_index': 16.6891...",5,43.936912,0.215000,0.401250,0.000000,0.350000,1
1,12.0,<p>I wound up using this. It is a kind of a ha...,"I wound up using this. It is a kind of a hack,...",1.0,"I wound up using this. It is a kind of a hack,...",32,3,(),"Doc(37 tokens; ""I wound up using this. It is a...","{'n_chars': 116, 'coleman_liau_index': 1.81524...",1,92.965000,0.286667,0.833333,0.108108,0.189189,1
2,28.0,"<p><a href=""http://www.codeproject.com/Article...",Oleg Shilo's C# Script solution (at The Code P...,0.0,Oleg Shilo's C# Script solution (at The Code P...,127,9,(),"Doc(152 tokens; ""Oleg Shilo's C# Script soluti...","{'n_chars': 664, 'coleman_liau_index': 11.9970...",25,53.334962,0.253333,0.583333,0.085526,0.138158,1
3,19.0,<p>I would be a bit reluctant to use nested cl...,I would be a bit reluctant to use nested class...,0.0,I would be a bit reluctant to use nested class...,127,6,(),"Doc(153 tokens; ""I would be a bit reluctant to...","{'n_chars': 620, 'coleman_liau_index': 10.4718...",18,55.041364,0.045000,0.475000,0.150327,0.104575,1
4,8.0,"<p>No, what you're doing is fine. Don't let th...","No, what you're doing is fine. Don't let those...",0.0,"No, what you're doing is fine. Don't let those...",106,9,(),"Doc(128 tokens; ""No, what you're doing is fine...","{'n_chars': 477, 'coleman_liau_index': 7.39185...",8,72.759286,0.314583,0.564583,0.093750,0.140625,1


## Get Textacy object

In [30]:
import textacy

In [31]:
combined['textacy'] = combined.spacy.apply(textacy.Doc)

## Get Summary column which contains all other attributes

In [32]:
combined['summary'] = combined.textacy.apply(textacy.text_stats.readability_stats)

In [31]:
combined.head()

,Score,Body,full_text,is_code_present,cleaned,no_of_words,no_of_sent,spacy,textacy,summary,no_of_polysyllable_words,flesch_readability,sentiment,subjectivity,quantitative,descriptive,good_bad
0,13.0,"<p><a href=""http://svnbook.red-bean.com/"">Vers...",Version Control with SubversionA very good res...,0.0,Version Control with SubversionA very good res...,17,2,(),"Doc(20 tokens; ""Version Control with Subversio...","{'n_chars': 104, 'coleman_liau_index': 16.6891...",5,43.936912,0.215000,0.401250,0.000000,0.350000,1
1,12.0,<p>I wound up using this. It is a kind of a ha...,"I wound up using this. It is a kind of a hack,...",1.0,"I wound up using this. It is a kind of a hack,...",32,3,(),"Doc(37 tokens; ""I wound up using this. It is a...","{'n_chars': 116, 'coleman_liau_index': 1.81524...",1,92.965000,0.286667,0.833333,0.108108,0.189189,1
2,28.0,"<p><a href=""http://www.codeproject.com/Article...",Oleg Shilo's C# Script solution (at The Code P...,0.0,Oleg Shilo's C# Script solution (at The Code P...,127,9,(),"Doc(152 tokens; ""Oleg Shilo's C# Script soluti...","{'n_chars': 664, 'coleman_liau_index': 11.9970...",25,53.334962,0.253333,0.583333,0.085526,0.138158,1
3,19.0,<p>I would be a bit reluctant to use nested cl...,I would be a bit reluctant to use nested class...,0.0,I would be a bit reluctant to use nested class...,127,6,(),"Doc(153 tokens; ""I would be a bit reluctant to...","{'n_chars': 620, 'coleman_liau_index': 10.4718...",18,55.041364,0.045000,0.475000,0.150327,0.104575,1
4,8.0,"<p>No, what you're doing is fine. Don't let th...","No, what you're doing is fine. Don't let those...",0.0,"No, what you're doing is fine. Don't let those...",106,9,(),"Doc(128 tokens; ""No, what you're doing is fine...","{'n_chars': 477, 'coleman_liau_index': 7.39185...",8,72.759286,0.314583,0.564583,0.093750,0.140625,1


## Save file to csv

In [8]:
#combined.to_csv("/Users/huangshanhongtao/Downloads/BT4222 Project Data/combined_clean.csv")

In [10]:
#combined = pd.read_csv("/Users/huangshanhongtao/Downloads/BT4222 Project Data/combined_clean.csv")

In [13]:
%store combined

Stored 'combined' (DataFrame)


## Convert Summary from string to dict

In [15]:
import ast

In [28]:
combined['summary'] = combined.summary.apply(ast.literal_eval)

## Get number_of_polysyllable_words

In [30]:
def get_no_of_polysyllable_words(data):
    return data.get("n_polysyllable_words")

In [31]:
combined['no_of_polysyllable_words'] = combined.summary.apply(get_no_of_polysyllable_words)

## Get flesch_readability

In [6]:
def get_flesch_readability(data):
    return data.get("flesch_readability_ease")

In [7]:
combined['flesch_readability'] = combined.summary.apply(get_flesch_readability)

## Get sentiment score by Textblob

In [14]:
from textblob import TextBlob

def get_textblob_sentiment(data):
    score = TextBlob(data)
    score = score.sentiment.polarity
    return score

In [15]:
combined['sentiment'] = combined.cleaned.apply(get_textblob_sentiment)

## Get subjectivity by Textblob

In [16]:
def get_textblob_subjectivity(data):
    score = TextBlob(data)
    score = score.sentiment.subjectivity
    return score

In [20]:
combined['subjectivity'] = combined.cleaned.apply(get_textblob_subjectivity)

In [60]:
combined.drop(['Unnamed: 0'], axis=1, inplace=True)

In [32]:
combined.head()

,Score,Body,full_text,is_code_present,cleaned,no_of_words,no_of_sent,spacy,textacy,summary,no_of_polysyllable_words,flesch_readability,sentiment,subjectivity,quantitative,descriptive,good_bad
0,13.0,"<p><a href=""http://svnbook.red-bean.com/"">Vers...",Version Control with SubversionA very good res...,0.0,Version Control with SubversionA very good res...,17,2,(),"Doc(20 tokens; ""Version Control with Subversio...","{'n_chars': 104, 'coleman_liau_index': 16.6891...",5,43.936912,0.215000,0.401250,0.000000,0.350000,1
1,12.0,<p>I wound up using this. It is a kind of a ha...,"I wound up using this. It is a kind of a hack,...",1.0,"I wound up using this. It is a kind of a hack,...",32,3,(),"Doc(37 tokens; ""I wound up using this. It is a...","{'n_chars': 116, 'coleman_liau_index': 1.81524...",1,92.965000,0.286667,0.833333,0.108108,0.189189,1
2,28.0,"<p><a href=""http://www.codeproject.com/Article...",Oleg Shilo's C# Script solution (at The Code P...,0.0,Oleg Shilo's C# Script solution (at The Code P...,127,9,(),"Doc(152 tokens; ""Oleg Shilo's C# Script soluti...","{'n_chars': 664, 'coleman_liau_index': 11.9970...",25,53.334962,0.253333,0.583333,0.085526,0.138158,1
3,19.0,<p>I would be a bit reluctant to use nested cl...,I would be a bit reluctant to use nested class...,0.0,I would be a bit reluctant to use nested class...,127,6,(),"Doc(153 tokens; ""I would be a bit reluctant to...","{'n_chars': 620, 'coleman_liau_index': 10.4718...",18,55.041364,0.045000,0.475000,0.150327,0.104575,1
4,8.0,"<p>No, what you're doing is fine. Don't let th...","No, what you're doing is fine. Don't let those...",0.0,"No, what you're doing is fine. Don't let those...",106,9,(),"Doc(128 tokens; ""No, what you're doing is fine...","{'n_chars': 477, 'coleman_liau_index': 7.39185...",8,72.759286,0.314583,0.564583,0.093750,0.140625,1


## Get how quantitative this answer is
### by calculating the percentage of NUM and DET words in the sentence

In [146]:
def get_quantitative(data):
    token_pos = [token.pos_ for token in data]
    counter = 0
    for i in range(0, len(token_pos)):
        if token_pos[i] == 'NUM' or token_pos[i] == 'DET':
            counter += 1
    
    if len(token_pos) == 0:
        return 0
    else:
        return (counter/len(token_pos))
    

In [147]:
combined['quantitative'] = combined.spacy.apply(get_quantitative)

## Get how descriptive the answer is
### by calculating the percentage of number of ADJ, ADV, ADP words in the sentence

In [243]:
def get_descriptive(data):
    token_pos = [token.pos_ for token in data]
    counter = 0
    for i in range(0, len(token_pos)):
        if token_pos[i] == 'ADJ' or token_pos[i] == 'ADV' or token_pos[i] == 'ADP':
            counter += 1
    
    if len(token_pos) == 0:
        return 0
    else:
        return (counter/len(token_pos))

In [244]:
combined['descriptive'] = combined.spacy.apply(get_descriptive)

In [81]:
combined.columns

Index(['Score', 'Body', 'full_text', 'is_code_present', 'cleaned',
       'no_of_words', 'no_of_sent', 'spacy', 'textacy', 'summary',
       'no_of_polysyllable_words', 'flesch_readability', 'no_of_syllable',
       'sentiment', 'subjectivity', 'quantitative', 'descriptive'],
      dtype='object')

## Transform Score into Good or Bad

In [245]:
combined['good_bad'] = 0

In [246]:
combined['good_bad'][combined.Score < 0] = -1

/Users/huangshanhongtao/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [247]:
combined['good_bad'][combined.Score >= 0] = 1

/Users/huangshanhongtao/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [248]:
combined.good_bad.value_counts()

 1    78112
-1    20981
Name: good_bad, dtype: int64

## Sampling

In [4]:
#sample = combined[(combined.no_of_sent >= 2) & ((combined.Score > 21) | (combined.Score < 0))].copy()
sample = combined[(combined.no_of_sent >= 1) & ((combined.Score > 21) | (combined.Score < 0))].copy()

In [64]:
sample[sample.Score < 0].shape

(20981, 18)

In [65]:
sample[sample.Score > 21].shape

(20728, 18)

In [37]:
sample.columns

Index(['Score', 'Body', 'full_text', 'is_code_present', 'cleaned',
       'no_of_words', 'no_of_sent', 'spacy', 'textacy', 'summary',
       'no_of_polysyllable_words', 'flesch_readability', 'no_of_syllable',
       'sentiment', 'subjectivity', 'quantitative', 'descriptive', 'good_bad'],
      dtype='object')

# Regression

In [14]:
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn import svm
from pandas.stats.api import ols

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
from sklearn import metrics

In [17]:
logistic = LogisticRegression()

In [33]:
# full parameters
X = sample[['is_code_present','no_of_words','no_of_sent','no_of_polysyllable_words', 'flesch_readability','sentiment','subjectivity','quantitative','descriptive']]
y = sample['good_bad']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
logistic_model = logistic.fit(X_train, y_train)

print('Score:')
print(logistic_model.score(X_test, y_test))
print('***************')
for i in range(0, len(X.columns)):
    print('Variable "{}" , Coefficient {}'.format(X.columns[i], logistic_model.coef_[0][i]))
    print('   ')
print('***************')
y_prob = logistic_model.predict_proba(X_test)[:,1]
y_predict = logistic_model.predict(X_test)

print('ROC-AUC Score:')
# roc auc score
print(metrics.roc_auc_score(y_test, y_prob))

Score:
0.668392788646
***************
Variable "is_code_present" , Coefficient 0.8422613084719808
   
Variable "no_of_words" , Coefficient 0.0007183756296657516
   
Variable "no_of_sent" , Coefficient 0.032950483375349046
   
Variable "no_of_polysyllable_words" , Coefficient 0.058022295579864006
   
Variable "flesch_readability" , Coefficient -0.0100376133465997
   
Variable "sentiment" , Coefficient 0.32540061364047657
   
Variable "subjectivity" , Coefficient 0.00794256220729872
   
Variable "quantitative" , Coefficient 3.155723947217872
   
Variable "descriptive" , Coefficient -0.413334171131617
   
***************
ROC-AUC Score:
0.722327107483


## Define a funtion to get the ROC-AUC score ---- Logistic Regression

In [12]:
def get_model_score(variable_list_input):
    X = sample[variable_list_input]
    y = sample['good_bad']
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
    logistic_model = logistic.fit(X_train, y_train)

    y_prob = logistic_model.predict_proba(X_test)[:,1]

    # roc auc score
    return metrics.roc_auc_score(y_test, y_prob)

## Generate different combination of the 9 parameters for comparison later

In [34]:
variable_list = ['is_code_present','no_of_words','no_of_sent','no_of_polysyllable_words', 'flesch_readability','sentiment','subjectivity','quantitative','descriptive']

In [35]:
from itertools import chain, combinations
def all_subsets(ss):
  return chain(*map(lambda x: combinations(ss, x), range(0, len(ss)+1)))

In [36]:
combination_list = []
for subset in all_subsets(variable_list):
    combination_list.append(list(subset))

In [37]:
# remove the first empty list
combination_list.remove([])

## Try all possible combination of variables, see which one performs the best

In [41]:
best_variable_combination = []
best_score = 0

for subset in combination_list:
    score = get_model_score(subset)
    if score > best_score:
        best_score = score
        best_variable_combination = subset
        print(best_score)
        print(best_variable_combination)

0.595816447062
['is_code_present']
0.668203964777
['no_of_words']
0.685160965699
['no_of_polysyllable_words']
0.703925565236
['is_code_present', 'no_of_words']
0.718793662513
['is_code_present', 'no_of_polysyllable_words']
0.718836000805
['is_code_present', 'no_of_words', 'no_of_polysyllable_words']
0.719135385131
['is_code_present', 'no_of_polysyllable_words', 'sentiment']
0.720191120421
['is_code_present', 'no_of_polysyllable_words', 'quantitative']
0.720249441326
['is_code_present', 'no_of_words', 'no_of_polysyllable_words', 'quantitative']
0.72027800404
['is_code_present', 'no_of_sent', 'no_of_polysyllable_words', 'quantitative']
0.720362441528
['is_code_present', 'no_of_polysyllable_words', 'flesch_readability', 'quantitative']
0.720967702541
['is_code_present', 'no_of_polysyllable_words', 'sentiment', 'quantitative']
0.720981790782
['is_code_present', 'no_of_words', 'no_of_polysyllable_words', 'flesch_readability', 'quantitative']
0.720998361938
['is_code_present', 'no_of_words',

## Therefore, the best combination of variables is:
### ['is_code_present', 'no_of_words', 'no_of_sent', 'no_of_polysyllable_words', 'flesch_readability', 'sentiment', 'quantitative', 'descriptive']

In [23]:
def get_model_full_summary(variable_list_input):
    X = sample[variable_list_input]
    y = sample['good_bad']
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
    logistic_model = logistic.fit(X_train, y_train)

    print('Score:')
    print(logistic_model.score(X_test, y_test))
    print('----------------')
    for i in range(0, len(X.columns)):
        print('Variable "{}" , Coefficient {}'.format(X.columns[i], logistic_model.coef_[0][i]))
        print('   ')
    print('----------------')
    y_prob = logistic_model.predict_proba(X_test)[:,1]
    y_predict = logistic_model.predict(X_test)

    print('ROC-AUC Score:')
    # roc auc score
    return metrics.roc_auc_score(y_test, y_prob)

In [40]:
get_model_full_summary(['is_code_present', 'no_of_words', 'no_of_sent', 'no_of_polysyllable_words', 'flesch_readability', 'sentiment', 'quantitative', 'descriptive'])

Score:
0.66915995397
----------------
Variable "is_code_present" , Coefficient 0.8323828992881231
   
Variable "no_of_words" , Coefficient 0.0006253912913383592
   
Variable "no_of_sent" , Coefficient 0.03387474209607688
   
Variable "no_of_polysyllable_words" , Coefficient 0.057904190686215475
   
Variable "flesch_readability" , Coefficient -0.010229774948124416
   
Variable "sentiment" , Coefficient 0.32564456016609555
   
Variable "quantitative" , Coefficient 3.7297905175844788
   
Variable "descriptive" , Coefficient -0.3187175312279934
   
----------------
ROC-AUC Score:


0.72215157461662316